# Dask Array Support to AnnData

## Initalizing

First let's do our imports and initalize adata objects with the help of the `adata_with_dask` function defined below.

In [1]:
import dask.array as da
import numpy as np
import pandas as pd
import anndata as ad

In [2]:
def adata_with_dask(M, N, c=50):
    adata_dict = {}
    adata_dict["X"] = da.random.random((M, N), chunks=c)
    adata_dict["dtype"] = np.float64
    adata_dict["obsm"] = dict(
        a=da.random.random((M, 100)),
    )
    adata_dict["layers"] = dict(
        a=da.random.random((M, N)),
    )
    adata_dict["obs"] = pd.DataFrame(
        {"batch": np.random.choice(["a", "b"], M)},
        index=[f"cell{i:03d}" for i in range(M)],
    )
    adata_dict["var"] = pd.DataFrame(index=[f"gene{i:03d}" for i in range(N)])
    
    
    return ad.AnnData(**adata_dict)

Here is how our adata looks like

In [3]:
adata = adata_with_dask(100,100)
adata

AnnData object with n_obs × n_vars = 100 × 100
    obs: 'batch'
    obsm: 'a'
    layers: 'a'

## Representation of Dask Arrays

Dask arrays consists of chunks that can be distributed in clusters. In the figure below, each small square represents a chunk that form a dask array. In principle these some of these chunks could be in different machines (clusters).

In [4]:
adata.X

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

In [5]:
adata.obsm['a']

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

## The Computation Graph
The graph layer in the `Count` row refers to the layer of the computation graph of the chunks, i.e. which operations are applied to them. We have this because the operations done on Dask arrays aren't computed instantly. This way, Dask array can optimize the queries we issued to it. It also won't keep our resources occupied for the results we expect later. Below is a representation of the chunks we initially created.

In [6]:
adata.X.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

We now show an example for this computation graph on dask arrays to understand it better. This part is not technically relevant to AnnData.

In [7]:
xsum = adata.X.sum(axis=1) # do a sum on axis=1
xsum

dask.array<sum-aggregate, shape=(100,), dtype=float64, chunksize=(50,), chunktype=numpy.ndarray>

Note that this computation isn't necessarily done yet, but rather saved to actually run it later.   If we investigate the computation graph of this result, we can see that for this operation some chunks aren't dependent on each other. This might give a hint to the Dask framework to store the chunks that depend on each other to the same cluster. For this simple exercise, all the chunks can be stored in one machine, but when it is impossible to store all the four chunks into one machine this will come in handy.

In [8]:
xsum.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

But coming back to our anndata tutorial we will see that nothing changed in our adata.

In [9]:
adata.X.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

## Concatenation

In this section we will cover how concatenation on anndata objects that use Dask arrays looks like. We first create another anndata object to concatenate with.

In [10]:
adata2 = adata_with_dask(100,100)
adata2

AnnData object with n_obs × n_vars = 100 × 100
    obs: 'batch'
    obsm: 'a'
    layers: 'a'

We can see that the X attribute of adata also consist of four chunks.

In [11]:
adata2.X

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

In [12]:
adata_concat = ad.concat([adata,adata2],index_unique='id')

When we concatenate the whole object you can see that in the X of the result consists of eight chunks and they quite probably are just the source chunks aligned.

In [13]:
adata_concat.X

dask.array<concatenate, shape=(200, 100), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

To confirm this we look at the computation graph. We can confirm that this new object's X is just the chunks of source X's put together.

In [14]:
adata_concat.X.visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

Here for the obsm we also this. The chunk from both are just stacked on top.

In [15]:
adata_concat.obsm['a']

dask.array<concatenate, shape=(200, 100), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [16]:
adata_concat.obsm['a'].visualize()

CytoscapeWidget(cytoscape_layout={'name': 'dagre', 'rankDir': 'BT', 'nodeSep': 10, 'edgeSep': 10, 'spacingFact…

## Views

Let's see how our views of anndata objects play with Dask arrays.

### Slice View



We take a slice of the concatenated adatas. This operation returns a view of the adata which means that the resulting adata holds a view of the source adata's Dask array, namely the `DaskArrayView` class,  which is a completely different object than Dask array.

In [17]:
adata_slice_view = adata_concat[:500, :][:, :500]

Below, you can see the values of the X attribute of the result. Which implies that the resulting object isn't "lazy" like Dask arrays. 

In [18]:
adata_slice_view.X, adata_slice_view.shape

(DaskArrayView([[0.53301795, 0.66787443, 0.15634334, ..., 0.32164397,
                 0.75044711, 0.71176344],
                [0.67788117, 0.48201184, 0.72151412, ..., 0.01904291,
                 0.45753518, 0.93067324],
                [0.17654758, 0.22333472, 0.43948775, ..., 0.25119521,
                 0.40568709, 0.9481696 ],
                ...,
                [0.80856321, 0.6160137 , 0.69223335, ..., 0.19004122,
                 0.33183754, 0.21153821],
                [0.65223497, 0.41924633, 0.5888846 , ..., 0.78818906,
                 0.51566138, 0.91042055],
                [0.36910591, 0.24653824, 0.05248607, ..., 0.48638579,
                 0.41379648, 0.22823851]]),
 (200, 100))

But our original adata remains unchanged.

In [19]:
adata_concat.X

dask.array<concatenate, shape=(200, 100), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

### Index List View

In [20]:
small_view = adata_concat[[12,12,3,5,53],[1,2,5]]
small_view

View of AnnData object with n_obs × n_vars = 5 × 3
    obs: 'batch'
    obsm: 'a'
    layers: 'a'

In [21]:
small_view.X

DaskArrayView([[0.10482385, 0.1220922 , 0.14001571],
               [0.10482385, 0.1220922 , 0.14001571],
               [0.78356573, 0.73719148, 0.74932912],
               [0.90583891, 0.06764586, 0.59375941],
               [0.17349612, 0.05048287, 0.69104416]])

### View by Category

In [22]:
categ_view = adata_concat[adata_concat.obs['batch'] == 'b']

In [23]:
categ_view.X

DaskArrayView([[0.53301795, 0.66787443, 0.15634334, ..., 0.32164397,
                0.75044711, 0.71176344],
               [0.67788117, 0.48201184, 0.72151412, ..., 0.01904291,
                0.45753518, 0.93067324],
               [0.87211266, 0.78356573, 0.73719148, ..., 0.76309687,
                0.44922404, 0.78628019],
               ...,
               [0.26697049, 0.44263729, 0.09259813, ..., 0.34704037,
                0.95992393, 0.08773989],
               [0.04647217, 0.39453262, 0.29082628, ..., 0.97541532,
                0.98596622, 0.23065648],
               [0.36910591, 0.24653824, 0.05248607, ..., 0.48638579,
                0.41379648, 0.22823851]])

## To Memory

In [24]:
adata_concat.X

dask.array<concatenate, shape=(200, 100), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

Here no copies are made, only the result of the lazy object is asked to be materialized.

In [25]:
adata_mem = adata_concat.to_memory(copy=False)
adata_mem.X

array([[0.53301795, 0.66787443, 0.15634334, ..., 0.32164397, 0.75044711,
        0.71176344],
       [0.67788117, 0.48201184, 0.72151412, ..., 0.01904291, 0.45753518,
        0.93067324],
       [0.17654758, 0.22333472, 0.43948775, ..., 0.25119521, 0.40568709,
        0.9481696 ],
       ...,
       [0.80856321, 0.6160137 , 0.69223335, ..., 0.19004122, 0.33183754,
        0.21153821],
       [0.65223497, 0.41924633, 0.5888846 , ..., 0.78818906, 0.51566138,
        0.91042055],
       [0.36910591, 0.24653824, 0.05248607, ..., 0.48638579, 0.41379648,
        0.22823851]])

If you want to both materialize the result and copy.

In [26]:
adata_mem = adata_concat.to_memory(copy=True)
adata_mem.X

array([[0.53301795, 0.66787443, 0.15634334, ..., 0.32164397, 0.75044711,
        0.71176344],
       [0.67788117, 0.48201184, 0.72151412, ..., 0.01904291, 0.45753518,
        0.93067324],
       [0.17654758, 0.22333472, 0.43948775, ..., 0.25119521, 0.40568709,
        0.9481696 ],
       ...,
       [0.80856321, 0.6160137 , 0.69223335, ..., 0.19004122, 0.33183754,
        0.21153821],
       [0.65223497, 0.41924633, 0.5888846 , ..., 0.78818906, 0.51566138,
        0.91042055],
       [0.36910591, 0.24653824, 0.05248607, ..., 0.48638579, 0.41379648,
        0.22823851]])

# IO operations

Read/Write operations on `h5ad` and `Zarr` are supported. One should note that the lazy objects are materialized when this is called. For now, the anndata loaded from file won't be loaded with dask arrays in it.

## Write h5ad

In [27]:
adata.write_h5ad('a1.h5ad')

In [28]:
adata.X

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

## Read h5ad

In [29]:
h5ad_adata = ad.read_h5ad('a1.h5ad')

In [30]:
h5ad_adata.X

array([[0.53301795, 0.66787443, 0.15634334, ..., 0.32164397, 0.75044711,
        0.71176344],
       [0.67788117, 0.48201184, 0.72151412, ..., 0.01904291, 0.45753518,
        0.93067324],
       [0.17654758, 0.22333472, 0.43948775, ..., 0.25119521, 0.40568709,
        0.9481696 ],
       ...,
       [0.37945035, 0.44935904, 0.87368098, ..., 0.5946007 , 0.44552927,
        0.67717767],
       [0.64752554, 0.90163149, 0.65256089, ..., 0.97912291, 0.07755674,
        0.49426686],
       [0.66668329, 0.97821502, 0.81251887, ..., 0.94387503, 0.74818926,
        0.31368062]])

## Write zarr

In [31]:
adata.write_zarr('a2.zarr')

In [32]:
adata.X

dask.array<random_sample, shape=(100, 100), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

## Read zarr

In [33]:
zarr_adata = ad.read_zarr('a2.zarr')

In [34]:
zarr_adata.X

array([[0.53301795, 0.66787443, 0.15634334, ..., 0.32164397, 0.75044711,
        0.71176344],
       [0.67788117, 0.48201184, 0.72151412, ..., 0.01904291, 0.45753518,
        0.93067324],
       [0.17654758, 0.22333472, 0.43948775, ..., 0.25119521, 0.40568709,
        0.9481696 ],
       ...,
       [0.37945035, 0.44935904, 0.87368098, ..., 0.5946007 , 0.44552927,
        0.67717767],
       [0.64752554, 0.90163149, 0.65256089, ..., 0.97912291, 0.07755674,
        0.49426686],
       [0.66668329, 0.97821502, 0.81251887, ..., 0.94387503, 0.74818926,
        0.31368062]])

Notice how they are loaded as arrays rather than dask arrays.

## Dask Array Support for Other Fields

This is the list of operations and in which fields they are supported, although some might have not been covered in this tutorial.

The following work with operations anndata supported before are also supported now with Dask arrays:
- anndata.concat()
- Views
- copy()
- to_memory() (changed behaviour)
- read/write on h5ad/zarr

**X, obsm, varm, obsp, varp, layers, uns, and raw** attributes are all supported and tested.

Note: scipy.sparse array wrapped with dask array doesn't play well. This is mainly because of the inconsistent numpy api support of scipy.sparse. Even though not explicitly tested, a sparse array that supports the numpy api should theoretically work well.